## A Julia package for evaluating the dilogarithm function

To use this package, you will need to put the files `polylog.jl`, `polylog_test.jl`, and `tests.jl` into a folder that Julia can locate. Once that is done, the command `using polylog` loads the package

In [2]:
using polylog

To run the accuracy tests in `polylog_test.jl`, manually include this file and run each of the five tests. Each test uses a dilogarithm function identity to compare the relative difference between values.  The test report gives the number of tests and the average scaled relative difference. The scaled relative difference is the relative difference divided by the machine epsilon. The report also shows the worst relative difference, and a dictionary that gives the number if tests a given scaled relative difference rounded to the nearest integer.

In [4]:
include(joinpath(dirname(pathof(polylog)), "polylog_test.jl"));

In [5]:
polylog2_test1(Complex{Float64},100,100)

number of tests = 10201
average error = 0.11293010489167729
worst error = 4


5-element Vector{Pair{Any, Any}}:
 0 => 9200
 1 => 875
 2 => 104
 3 => 19
 4 => 3

In [6]:
polylog2_test2(Complex{Float64},10^6)

number of tests = 1000000
average error = 0.461617
worst error = 9


10-element Vector{Pair{Any, Any}}:
 0 => 662502
 1 => 248081
 2 => 63495
 3 => 19075
 4 => 5284
 5 => 1239
 6 => 285
 7 => 33
 8 => 5
 9 => 1

In [7]:
polylog2_test2(Complex{Float64},10^6)

number of tests = 1000000
average error = 0.461617
worst error = 9


10-element Vector{Pair{Any, Any}}:
 0 => 662502
 1 => 248081
 2 => 63495
 3 => 19075
 4 => 5284
 5 => 1239
 6 => 285
 7 => 33
 8 => 5
 9 => 1

In [8]:
polylog2_test3(Complex{Float64},10^3,4)

number of tests = 999999
average error = 2.0367670367670367
worst error = 22


22-element Vector{Pair{Any, Any}}:
  0 => 289123
  1 => 220676
  2 => 163889
  3 => 116974
  4 => 80350
  5 => 52475
  6 => 32644
  7 => 19568
  8 => 11200
  9 => 6272
 10 => 3417
 11 => 1746
 12 => 896
 13 => 432
 14 => 191
 15 => 90
 16 => 28
 17 => 14
 18 => 5
 19 => 5
 21 => 1
 22 => 3

In [9]:
polylog2_test4(Float64,10^6)

number of tests = 1000000
average error = 0.922635
worst error = 7


8-element Vector{Pair{Any, Any}}:
 0 => 453180
 1 => 315905
 2 => 128403
 3 => 68145
 4 => 27329
 5 => 6115
 6 => 863
 7 => 60

In [10]:
polylog2_test5(Float64,10^3)

number of tests = 1000000
average error = 0.486234
worst error = 8


9-element Vector{Pair{Any, Any}}:
 0 => 644443
 1 => 257916
 2 => 72530
 3 => 18836
 4 => 4963
 5 => 1026
 6 => 240
 7 => 40
 8 => 6

Here we run the unit tests--for the details of what each test does, you'll need to read the source code.

In [11]:
include(joinpath(dirname(pathof(polylog)), "tests.jl"));


Special Values Test
Test Summary: | Pass  Total  Time
test set      |    3      3  0.5s

Binary 16 Tests
Test Summary: | Pass  Total  Time
test set      |   17     17  0.7s

Binary32 Tests
Test Summary: | Pass  Total  Time
test set      |   17     17  0.4s

Binary64 Tests
Test Summary: | Pass  Total  Time
test set      |   17     17  0.1s

BigFloat Tests
Test Summary: | Pass  Total  Time
test set      |   17     17  0.9s

Table 27.7 Abramowitz & Stegun
Test Summary: | Pass  Total  Time
test set      |   51     51  0.1s

Test DLMF identity 25.12.3E3
Test Summary: | Pass  Total   Time
test set      |    4      4  50.9s

Test DLMF identity 25.12.E5
Test Summary: | Pass  Total   Time
test set      |    4      4  49.9s

Test DLMF identity 25.12.E7
Test Summary: | Pass  Total   Time
test set      |    4      4  37.5s

Test Dilogarithm reflection identity
Test Summary: | Pass  Total     Time
test set      |    4      4  1m24.1s

Compare to PolyLog
Test Summary: | Pass  Total   Time
test set 

In [12]:
function rd(a::Real,b::Real)
    if isinf(a) || isinf(b) || isnan(a) || isnan(b)
       Inf
    else
     abs(a-b)/(min(abs(a),abs(b)))
    end
 end


rd (generic function with 2 methods)

In [13]:
 function rd(a::Number,b::Number)
      max(rd(real(a),real(b)), rd(imag(a),imag(b)))
 end

rd (generic function with 2 methods)

In [14]:
function relativeDiff(x,exact)
    rr = abs(real(x)-real(exact))/abs(real(exact))
    ri = abs(imag(x)-imag(exact))/abs(imag(exact))    
    max(if isnan(rr) 0 else rr end,  if isnan(ri) 0 else ri end)       
end

relativeDiff (generic function with 1 method)

In [15]:
function compare_polylog2(T::Type, n)
    mm = -Inf
    xxx = 0
    pie = convert(BigFloat,pi)
    for i = 1 : n
        for j = 0 : n-1
            x = (i/n) * cis(2*pie* j/n)
            m = relativeDiff(polylog2(x), polylog2(convert(Complex{T},x)))
            if m > mm
                xxx = x
                mm = m
            end
        end
    end
  convert(Float64,mm), convert(Complex{Float64}, xxx)
end

compare_polylog2 (generic function with 2 methods)

In [16]:
compare_polylog2(Float64, 50)

(9.639329568507601e-15, 0.13866217279343623 + 0.7268925655392297im)

In [17]:
function multiple_prec(x)
    y16 = polylog2(convert(Complex{Float16},x))
    y32 = polylog2(convert(Complex{Float32},x))
    y64 = polylog2(convert(Complex{Float64},x))
    ybf = polylog2(convert(Complex{BigFloat},x))
    isapprox(y16, convert(Complex{Float16},y32), atol=8*eps(Float16)) &&
    isapprox(y32, convert(Complex{Float32},y32), atol=4*eps(Float32)) &&
    isapprox(y64, convert(Complex{Float64},ybf), atol=4*eps(Float64))
end

multiple_prec (generic function with 1 method)

In [18]:
multiple_prec(28*im)

true

I asked chatGPT 3.5 "Write accurate Julia code that evaluates the dilogarithm function."  This is what I got

In [19]:
using QuadGK

In [20]:
using BenchmarkTools

In [21]:
function dilogarithm_chatGPT(z::Number)
    if z == 1.0
        return π^2 / 6
    elseif abs(z) > 1
        return real(dilogarithm_chatGPT(complex(z)))
    end

    f(t) = log(1 - t) / t
    result, err = quadgk(f, 0, z)
    return -result
end

dilogarithm_chatGPT (generic function with 1 method)

For inputs outside the unit circle, the function `dilogarithm_chatGPT` is broken. Likely, just removing the `elseif abs(z) > 1` will fix this error.

Let's do some timings to compare speeds. With one test, they have pretty much the same speed

In [22]:
q1 = @btime dilogarithm_chatGPT(cis(2*pi/3))

  1.080 μs (0 allocations: 0 bytes)


-0.5483113556160755 + 0.6766277376064359im

In [23]:
q2 = @btime polylog2(cis(2*pi/3))

  1.670 μs (0 allocations: 0 bytes)


-0.5483113556160752 + 0.6766277376064358im

For this case, the two methods have good agreement

In [25]:
q1-q2

-2.220446049250313e-16 + 1.1102230246251565e-16im

In [85]:
function boa(x::Real,y::Real)
    ex = exponent(x)
    ey = exponent(y)
    sx = significand(x)
    sy = significand(y)
    if ex !== ey || typeof(x) !== typeof(y)
        0
    elseif x==y
        precision(x)
    else
        -exponent(sx-sy)
    end
end

boa (generic function with 1 method)

In [101]:
boa(2.0, 1.999)

0

In [89]:
precision(3.4)

53